# Image Transforms Tutorial

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O tutorial_image.jpg

In [ ]:
image_bgr = cv2.imread("./tutorial_image.jpg")
image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
print(image.shape)
plt.title("Original Image")
plt.imshow(image);

## Crop

:information_source: All transforms leave the original image unmodified.

In [ ]:
from airo_camera_toolkit.image_transforms import Crop
# from airo_camera_toolkit.image_transforms import *

crop = Crop(image.shape, x=160, y=120, h=200, w=200)

cropped_image = crop(image)

plt.title("Cropped Image")
plt.imshow(cropped_image);

### Transforming Points

The `ImageTransform` classes also provide functionality to transform points from the original image to the transformed image, and in reverse.
This is useful e.g. when running a keypoint detector on a transformed image and wanting to map the detected keypoints back to the original image. 

In [ ]:
image1 = image.copy()

point = (311, 150) # tip of the feather

cv2.circle(image1, point, 9, (0, 255, 0), thickness=2)

plt.title("Point in Original Image (green)")
plt.imshow(image1);

In [ ]:
point_in_crop = crop.transform_point(point)
point_in_crop

In [ ]:
cv2.circle(cropped_image, point_in_crop, 9, (0, 0, 255), thickness=2)

plt.title("Point from Original Transformed to Crop (blue)")
plt.imshow(cropped_image);

In [ ]:
point1_in_crop = (18, 125) # tip of the horse's ear

cv2.circle(cropped_image, point1_in_crop, 9, (255, 0, 0), thickness=2)

plt.title("Point in Cropped Image")
plt.imshow(cropped_image);

In [ ]:
point1_in_original = crop.reverse_transform_point(point1_in_crop)

cv2.circle(image1, point1_in_original, 9, (255, 255, 0), thickness=2)
plt.title("Point from Crop Transformed to Original (yellow)")
plt.imshow(image1);

## Resize

In [ ]:
from airo_camera_toolkit.image_transforms import Resize

height, width, _ = image.shape
height_new, width_new = height // 4, width // 4

resize = Resize(image.shape, h=height_new, w=width_new)

resized_image = resize(image)
print(resized_image.shape)

plt.imshow(resized_image);

In [ ]:
point_in_resized = resize.transform_point(point)
point_in_resized

In [ ]:
cv2.circle(resized_image, point_in_resized, 4, (0, 0, 255), thickness=1)
plt.imshow(resized_image);

:information_source: By default, transformed points are rounded to the nearest integer pixel coordinate. To disable this and remove rounding error, set `round_transformed_points=False`.

In [ ]:
resize2 = Resize(image.shape, h=height_new, w=width_new, round_transformed_points=False)
point_in_resized2 = resize2.transform_point(point)
point_in_resized2

In [ ]:
point_reverse_resized = resize.reverse_transform_point(point_in_resized)
point, point_reverse_resized

In [ ]:
point_reverse_resized2 = resize2.reverse_transform_point(point_in_resized2)
point_reverse_resized2_int = tuple(map(int, point_reverse_resized2))
point, point_reverse_resized2, point_reverse_resized2_int

In [ ]:
image2 = image.copy()
cv2.circle(image2, point_reverse_resized, 9, (255, 0, 0), thickness=2)
cv2.circle(image2, point_reverse_resized2_int, 9, (0, 255, 0), thickness=2)
plt.imshow(image2);

## 90-degree Rotations

In [ ]:
from airo_camera_toolkit.image_transforms import Rotation90

rotation90 = Rotation90(input_shape=image.shape)
image_rotated = rotation90(image)
plt.imshow(image_rotated);

In [ ]:
point_in_rotated = rotation90.transform_point(point)

cv2.circle(image_rotated, point_in_rotated, 9, (255, 0, 0), thickness=2)
plt.imshow(image_rotated);

In [ ]:
rotation180 = Rotation90(image.shape, 2)
image_rotated180 = rotation180(image)
point_in_rotated180 = rotation180.transform_point(point)
cv2.circle(image_rotated180, point_in_rotated180, 9, (0, 255, 0), thickness=2)
plt.imshow(image_rotated180);

In [ ]:
rotation270 = Rotation90(image.shape, 3)
image_rotated270 = rotation270(image)
point_in_rotated270 = rotation270.transform_point(point)
cv2.circle(image_rotated270, point_in_rotated270, 9, (0, 0, 255), thickness=2)
plt.imshow(image_rotated270);

In [ ]:
image3 = image.copy()

point_in_rotated90_reverse = rotation90.reverse_transform_point(point_in_rotated)
point_in_rotated180_reverse = rotation180.reverse_transform_point(point_in_rotated180)
point_in_rotated270_reverse = rotation270.reverse_transform_point(point_in_rotated270)

cv2.circle(image3, point_in_rotated90_reverse, 16, (255, 0, 0), thickness=2)
cv2.circle(image3, point_in_rotated180_reverse, 12, (0, 255, 0), thickness=2)
cv2.circle(image3, point_in_rotated270_reverse, 8, (0, 0, 255), thickness=2)

plt.imshow(image3);


## Composing Transforms

In [ ]:
crop = Crop(image.shape, x=160, y=120, h=200, w=200)
resize = Resize(crop.shape, h=100, w=100, round_transformed_points=False)
rotation_clockwise = Rotation90(resize.shape, -1)

image_transformed = rotation_clockwise(resize(crop(image)))
plt.imshow(image_transformed);

In [ ]:
from airo_camera_toolkit.image_transforms import ComposedTransform

transform = ComposedTransform([crop, resize, rotation_clockwise])
image_transformed2 = transform(image)
point_transformed2 = transform.transform_point(point)


print(point)
print(point_transformed2)

point_transformed2_int = tuple(map(int, point_transformed2))

cv2.circle(image_transformed2, point_transformed2_int, 5, (0, 255, 0), thickness=1)

plt.imshow(image_transformed2);


In [ ]:
image4 = image.copy()

point_transformed2_reverse = transform.reverse_transform_point(point_transformed2)
print(point_transformed2_reverse)

point_transformed2_reverse_int = tuple(map(int, point_transformed2_reverse))

cv2.circle(image4, point_transformed2_reverse_int, 9, (0, 255, 0), thickness=2)
plt.imshow(image4);